## S-Mobile: Predicting Customer Churn-- Questions

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyrsm as rsm
import seaborn as sns
import statsmodels.formula.api as smf
import xgboost as xgb
from sklearn import metrics, preprocessing
from sklearn.inspection import permutation_importance, plot_partial_dependence
from statsmodels.genmod.families import Binomial
from statsmodels.genmod.families.links import logit

mpl.rcParams["figure.dpi"] = 150

import warnings
warnings.filterwarnings("ignore")
plt.ioff()

# increase plot resolution
# mpl.rcParams["figure.dpi"] = 150

In [2]:
## load the data - this dataset must NOT be changed
s_mobile = pd.read_pickle("data/s_mobile.pkl")
s_mobile["churn_yes"] = (s_mobile["churn"] == "yes").astype(int)

If you want access to the full 5M row dataset, use the dropbox link below.

https://www.dropbox.com/s/eli6a1q6qisacci/s_mobile_1M.pkl?dl=1

The downside to using the dataset with 1M rows is, of course, that estimation time will increase substantially. I do NOT recommend you use this dataset to select your final model or for tuning hyper parameters. You can, however, use this larger dataset to re-estimate your chosen model and generate profit estimates for the representative sample.

In [3]:
# show dataset description
#rsm.describe(s_mobile)

Use `smf.glm` with `freq_weights` and `cov_type` like in the below example
    
```python
lr = smf.glm(
    formula="churn_yes ~ changer + changem + ...",
    family=Binomial(link=logit()),
    data=pentathlon_nptb.query("training == 1"),
    freq_weights=s_mobile.loc[mobile.training == 1, "cweight"],
).fit(cov_type="HC1")
```

## EDA

In [4]:
# explore training set values
training_eda = pd.get_dummies(
    s_mobile.loc[s_mobile.training == 1, "changer":],
).agg(["mean", "std", "min", "max"]).T

In [5]:
# churn in representative model
rep_eda = pd.get_dummies(
    s_mobile.loc[s_mobile.representative == 1, "changer":],
).agg(["mean", "std", "min", "max"]).T

In [6]:
s_mobile[s_mobile["representative"] == 1].churn.value_counts()

no     29400
yes      600
Name: churn, dtype: int64

In the representative data, since we observe a difference of 294:6 in the churn_yes and churn_no rates, the case weights will be 294:6 (reduced to 49:1).

In [7]:
# add case weights
class_weight_dict = {0: 49, 1: 1}
s_mobile["churn_yes"] = (s_mobile["churn"] == "yes").astype(int)
s_mobile["cweight"] = rsm.ifelse(
    s_mobile.churn == "yes", class_weight_dict[1], class_weight_dict[0]
)

In [8]:
eda_plots = rsm.distr_plot(s_mobile.loc[s_mobile.training == 1, "changer":"occupation"])

## Estimate Logistic Regression

In [9]:
idvar = "customer"
rvar = "churn_yes"
evar = [
    "refurb",
    "smartphone",
    "highcreditr",
    "mcycle",
    "car",
    "travel",
    "region",
    "occupation",
    "changer",
    "changem",
    "revenue",
    "mou",
    "overage",
    "roam",
    "conference",
    "months",
    "uniqsubs",
    "custcare",
    "retcalls",
    "dropvce",
    "eqpdays",
]

Logistic regression with case weights, as specified above, but no standardization.

In [10]:
form = "churn_yes ~ " + " + ".join(evar)
lr_mod = smf.glm(
    formula=form,
    family=Binomial(link=logit()),
    freq_weights=s_mobile.loc[s_mobile.training == 1, "cweight"],
    data=s_mobile[s_mobile.training == 1],
)
lr = lr_mod.fit(cov_type="HC1")
rsm.or_ci(lr)

,index,OR,OR%,2.5%,97.5%,p.values,
1,refurb[T.yes],1.318,31.8%,1.255,1.385,< .001,***
2,smartphone[T.yes],0.913,-8.7%,0.860,0.969,0.003,**
3,highcreditr[T.yes],0.483,-51.7%,0.455,0.513,< .001,***
4,mcycle[T.yes],0.883,-11.7%,0.834,0.934,< .001,***
5,car[T.yes],0.991,-0.9%,0.919,1.068,0.809,
6,travel[T.yes],0.781,-21.9%,0.723,0.844,< .001,***
7,region[T.NE],0.644,-35.6%,0.611,0.678,< .001,***
8,region[T.NW],0.660,-34.0%,0.627,0.695,< .001,***
9,region[T.SE],0.651,-34.9%,0.618,0.685,< .001,***
10,region[T.SW],0.627,-37.3%,0.595,0.660,< .001,***


In [11]:
or_un = rsm.or_plot(lr)
or_un = or_un.set_xlabel("Odds-ratio (un-standardized)")

#### Logistic Regression model with case weights and standardization

In [12]:
Xs = rsm.scale_df(
    s_mobile.loc[:, "changer":"occupation"],
    wt=s_mobile.cweight,
    train=s_mobile.training == 1,
)
Xs["churn_yes"] = s_mobile.churn_yes
# Xs

In [13]:
lr_std = smf.glm(
    formula=form,
    family=Binomial(link=logit()),
    freq_weights=s_mobile.loc[s_mobile.training == 1, "cweight"],
    data=Xs[s_mobile.training == 1],
).fit(cov_type="HC1")

In [14]:
importances = (
    rsm.or_ci(lr_std, importance=True, data=s_mobile[s_mobile.training == 1])
    .sort_values("importance", ascending=False)
    .reset_index(drop=True)
)

`roam`, `uniqsubs` and `changer` have OR of exactly 1.

In [15]:
# plot size
or_std = rsm.or_plot(lr_std, figsize=(6, 6))
or_std = or_std.set_xlabel("Odds-ratio (standardized)")

# fig = rsm.or_plot(lr_std)
# fig = fig.set_xlabel("Odds-ratio (standardized)")

Changes to see a flipped-OR plot

In [16]:
# flipping the OR plot: this flipping is optional and we probably should not use it since it is difficutlt to interpret 1/2 flipped and the other 1/2 as is
# s_mobile_flipped = Xs.copy()
# s_mobile_flipped["changem"] = -s_mobile_flipped.changem
# s_mobile_flipped["revenue"] = -s_mobile_flipped.revenue
# s_mobile_flipped["months"] = -s_mobile_flipped.months
# s_mobile_flipped["mou"] = -s_mobile_flipped.mou
# s_mobile_flipped["conference"] = -s_mobile_flipped.conference
# s_mobile_flipped["custcare"] = -s_mobile_flipped.custcare

# s_mobile_flipped["smartphone"] = rsm.ifelse(s_mobile_flipped.smartphone == "yes", "no", "yes")
# s_mobile_flipped["highcreditr"] = rsm.ifelse(s_mobile_flipped.highcreditr == "yes", "no", "yes")
# s_mobile_flipped["mcycle"] = rsm.ifelse(s_mobile_flipped.mcycle == "yes", "no", "yes")
# s_mobile_flipped["travel"] = rsm.ifelse(s_mobile_flipped.travel == "yes", "no", "yes")
# s_mobile_flipped["car"] = rsm.ifelse(s_mobile_flipped.car == "yes", "no", "yes")

# lr_flip = smf.glm(
#     formula=form,
#     family=Binomial(link=logit()),
#     freq_weights=s_mobile.loc[s_mobile.training == 1, "cweight"],
#     data=s_mobile_flipped[s_mobile.training == 1],
# ).fit(cov_type="HC1")
# # lr.summary()
# fig = rsm.or_plot(lr_flip)
# fig = fig.set_xlabel("Odds-ratio (standardized)")

In [17]:
wald_table = lr_std.wald_test_terms().table.round(3)

## Alternative model: XGBoost

In [18]:
X = pd.get_dummies(s_mobile.loc[:, "changer":"occupation"], drop_first=True)
y = s_mobile.churn_yes

In [19]:
clf = xgb.XGBClassifier(
    max_depth=2,
    n_estimators=100,
    objective="binary:logistic",
    use_label_encoder=False,
    scale_pos_weight=1 / 49,
    eval_metric="auc",
    random_state=1234,
).fit(X[s_mobile.training == 1], y[s_mobile.training == 1], verbose=True)

In [20]:
from sklearn.model_selection import GridSearchCV
param_grid = {
"n_estimators": [50, 100, 150],
"scale_pos_weight":[1/49],
'use_label_encoder':[False],
"max_depth": [2,4],
'random_state':[1234],
}
scoring = {"AUC": "roc_auc"}
# clf_cv = GridSearchCV(
# clf, param_grid, scoring=scoring, cv=5, n_jobs=4, refit="AUC", verbose=True
# ).fit(X[s_mobile.training == 1], y[s_mobile.training == 1])

In [21]:
act_pred = (600 / 29400)
lr_pred = (lr.predict(s_mobile[s_mobile.representative == 1]).mean().round(5))
clf_pred = (clf.predict_proba(X[s_mobile.representative == 1])[:, 1].mean().round(5))
#cv_pred = (clf_cv.predict_proba(X[s_mobile.representative == 1])[:, 1].mean().round(5))

In [22]:
preds = [act_pred, lr_pred, clf_pred]#, cv_pred]

The two values above are close enough for us to conclude that predicted probabilities in the representative dataset are scaled appropriately

In [23]:
def importance(clf, X, y, cn):
    imp = permutation_importance(
        clf, X, y, scoring="roc_auc", n_repeats=10, random_state=1234
    )
    data = pd.DataFrame(imp.importances.T)
    data.columns = cn
    order = data.agg("mean").sort_values(ascending=False).index
    fig = sns.barplot(
        x="value", y="variable", color="slateblue", data=pd.melt(data[order])
    )
    fig.set(title="Permutation Importances", xlabel=None, ylabel=None)
    return fig

In [24]:
clf_imp = importance(clf, X[s_mobile.training == 1], y[s_mobile.training == 1], X.columns)
#cv_imp = importance(clf_cv, X[s_mobile.training == 1], y[s_mobile.training == 1], X.columns)

In [25]:
fig, ax = plt.subplots(figsize=(6, 3))
features=[4,12]
ax.set_title("Partial Dependence Plots")
fig = plot_partial_dependence(
    clf,
    X[s_mobile.training == 1], features,
    ax=ax,
)

Adding nonlinear terms in XGBoost is not needed probably

### Alternate Model: Tuned Logistic Regression for Non-Linearity

In [26]:
Xs['ov_2'] = Xs['overage'] ** 2
Xs['ov_cubed'] = Xs['overage'] * Xs['overage'] * Xs['overage']

s_mobile['ov_cubed'] = s_mobile['overage'] * s_mobile['overage'] * s_mobile['overage']

form_tune = 'churn_yes ~ refurb + smartphone + highcreditr + mcycle + car + travel + region + occupation + changer + changem\
+ revenue + mou + overage + roam + conference + months + uniqsubs + custcare + retcalls + dropvce + eqpdays + ov_2 + ov_cubed'


# lr_tune_mod = smf.glm(
#     formula=form_tune,
#     family=Binomial(link=logit()),
#     freq_weights=s_mobile.loc[s_mobile.training == 1, "cweight"],
#     data=s_mobile[s_mobile.training == 1],
# )
# lr_tune = lr_tune_mod.fit(cov_type="HC1")

lr_tune_std = smf.glm(
    formula=form_tune,
    family=Binomial(link=logit()),
    freq_weights=s_mobile.loc[s_mobile.training == 1, "cweight"],
    data=Xs[s_mobile.training == 1],
).fit(cov_type="HC1")
#rsm.or_ci(lr_tune)

importances_alt = (
    rsm.or_ci(lr_tune_std, importance=True, data=Xs[s_mobile.training == 1])
    .sort_values("importance", ascending=False)
    .reset_index(drop=True)
)

In [27]:
# x = importances['OR']
# y = importances_tuned['OR']

# combined = pd.DataFrame([x,y])
# combined

In [28]:
# x1 = importances['importance']
# x2 = importances_tuned['importance']

# imp = pd.DataFrame([x1,x2])
# imp

In [29]:
# lr = smf.glm(
#     formula=form,
#     family=Binomial(link=logit()),
#     freq_weights=s_mobile.loc[s_mobile.training == 1, "cweight"],
#     data=Xs[s_mobile.training == 1],
# )

# rsm.vif(lr)

### Actions/Offers/Incentives

Assumption: overage leads to overage fee for the customers

In [30]:
s_mobile_un = s_mobile.query("representative == 1").copy()
perc = s_mobile_un[s_mobile_un.overage < 100]['overage'].sum()/s_mobile_un[s_mobile_un.overage < 100]['mou'].sum()

s_mobile_un['og_rev'] = s_mobile_un['revenue'].copy()
s_mobile_un['revenue'] = s_mobile_un['revenue'] * (1-perc)

### CLV Calculations Function

In [31]:
def clv(revenue,hike,ser_mar_cost,n_months,churn): 
    d1 = {
    "Offer CLV": [
        "Revenue",
        "Service & Marketing cost",
        "Customer Profit",
        "Churn rate",
        "Prob.of being active at end of period",
        "Profit expected on average",
        "Discount",
        "Present value of expected profits",
        "CLV",
    ],
    "Today": [0] * 9,
    }
    d2 = ["Month %i" % i for i in range(1, n_months+1)]
    df1 = pd.DataFrame(data=d1)
    df2 = pd.DataFrame(0, index=np.arange(9), columns=d2)
    apc = pd.concat([df1, df2], axis=1).set_index("Offer CLV")

    # Revenue
    apc.loc["Revenue"] = apc.loc["Revenue"].astype(float)
    apc.loc["Revenue"]["Month 1":"Month 12"] = revenue
    for i in range(13, n_months+1):
        p = "Month %i" % i
        q = "Month %i" % (i - 12)
        apc.loc["Revenue"][p] = apc.loc["Revenue"][q] * (hike+1)

    # Service cost
    apc.loc["Service & Marketing cost"] = apc.loc["Revenue"] * ser_mar_cost

    # Customer Profit
    apc.loc["Customer Profit"] = (apc.loc["Revenue"] - apc.loc["Service & Marketing cost"])

    # Churn
#     nc = [churn]*int((n_months/12))
    apc.loc["Churn rate"]["Today"] = 1
    apc.loc["Churn rate"] = churn

#     for i in range(1, len(nc)):
#         j = 1 + i * 12
#         p = "Month %i" % j
#         apc.loc["Churn rate"][p] = nc[i]

    # Prob of being active at the end of the period
    apc.loc["Prob.of being active at end of period"]["Today"] = 1
    apc.loc["Prob.of being active at end of period"]["Month 1"] = apc.loc["Prob.of being active at end of period"]["Today"] * (1 - apc.loc["Churn rate"]["Month 1"])
    for i in range(3, (n_months+2)):
        p = "Month %i" % (i - 1)
        q = "Month %i" % (i - 2)
        apc.loc["Prob.of being active at end of period"][p] = apc.loc["Prob.of being active at end of period"][q] * (1 - apc.loc["Churn rate"][p])

    # Profit expected on average
    apc.loc["Profit expected on average"] = (apc.loc["Customer Profit"] * apc.loc["Prob.of being active at end of period"])

    # Discount
    apc.loc["Discount"] = list(range(0, (n_months+1)))
    apc.loc["Present value of expected profits"] = (apc.loc["Profit expected on average"] / (1.00797414) ** apc.loc["Discount"])
    apc.loc["CLV"]["Month 1"] = apc.loc["Present value of expected profits"]["Month 1"]
    for i in range(3, (n_months+2)):
        p = "Month %i" % (i - 1)
        q = "Month %i" % (i - 2)
        apc.loc["CLV"][p] = (apc.loc["CLV"][q] + apc.loc["Present value of expected profits"][p])
    CLV_apc = apc.round(decimals=3)
    return CLV_apc

### Plan 1: subsidize overage costs

For customers that have overage less than 100 mins, their overage fee is subsidized by the company.
For customers that have overage greater than 100 mins, their overage is subsidized by 100 mins (at 1/2 the overage rate = 5 cents), and the remaining overage is charged the same (10cents).

In [32]:
s_mobile_un["p_og"] = lr.predict(s_mobile_un)

In [33]:
#plan: for customers that don't cross 100 mins in overage, their overage is free
s_mobile_un["p_overage1_2"] = lr.predict(s_mobile_un[(s_mobile_un.overage <100)].assign(overage = 0))

s_mobile_un[(s_mobile_un.overage < 100)].loc[s_mobile_un.overage > 0, ["churn_yes", "p_overage1_2"]].agg(
    ["count", "mean"]
).round(4)

,churn_yes,p_overage1_2
count,5787.0000,5787.0000
mean,0.0299,0.0156


In [34]:
cost = (s_mobile_un[(s_mobile_un.overage <100)]['overage'].sum() * 0.1)/5787

og_sub_rev = s_mobile_un[(s_mobile_un.overage <100)].og_rev.mean()
sub_rev = og_sub_rev - cost 
#cost_perc_1 = cost/sub_rev

p1_b = (clv(og_sub_rev,0,0,60,0.0299)[-1:]['Month 60']) #before
p1_a = (clv(sub_rev,0,0,60,0.0156)[-1:]['Month 60']) #after

In [35]:
p1_a - p1_b

Offer CLV
CLV    177.263
Name: Month 60, dtype: float64

#### Plan 1: part 2

In [36]:
s_what = s_mobile_un.copy()
s_what['overage'] = s_what['overage'] - 100
#s_what.overage = s_what.overage.clip(0,165)
#s_what[s_what.overage > 100].overage.value_counts()

In [37]:
s_what["p_overage1_2n"] = lr.predict(s_what[(s_mobile_un.overage >100)])

s_what[(s_mobile_un.overage > 100)].loc[s_mobile_un.overage > 0, ["churn_yes", "p_overage1_2n"]].agg(
    ["count", "mean"]
).round(4)

,churn_yes,p_overage1_2n
count,10335.0000,10335.0000
mean,0.0215,0.0155


In [38]:
sub_rev_3= s_what[s_mobile_un.overage>100].revenue.mean()
og_rev_3= s_what[s_mobile_un.overage>100].og_rev.mean()

cost_3 = 100 * 0.05
cost_perc_3 = cost_3 /sub_rev_3

p1b_b = (clv(og_rev_3,0,0.1,60,0.0215)[-1:]['Month 60']) #before
p1b_a =(clv(sub_rev_3,0,cost_perc_3,60,0.0155)[-1:]['Month 60']) #after

p1b_a - p1b_b

Offer CLV
CLV    296.394
Name: Month 60, dtype: float64

### PLAN 2

In [39]:
s_mobile_2 = s_mobile_un.copy()
s_mobile_2['ovs'] = np.where(s_mobile_2.mou != 0, s_mobile_2['overage'] / s_mobile_2['mou'], s_mobile_2.overage) #accounting for customers who have 0 mou and overage
#test.ovs.max()

In [40]:
#plan: for customers that don't cross 100 mins in overage, their overage is free
s_mobile_2["p_overage1_3"] = lr.predict(s_mobile_2[(s_mobile_2.ovs > 0) &(s_mobile_2.ovs <0.10)].assign(overage = 0))

s_mobile_2[(s_mobile_2.ovs > 0) &(s_mobile_2.ovs < 0.10)].loc[s_mobile_2.overage > 0, ["churn_yes", "p_overage1_3"]].agg(
    ["count", "mean"]
).round(4)

,churn_yes,p_overage1_3
count,3415.0000,3415.0000
mean,0.0176,0.0112


In [41]:
cost_3 = (s_mobile_2[(s_mobile_2.ovs >0) & (s_mobile_2.ovs <0.10)]['overage'].sum() * 0.1)/3415
og_sub_rev_3 = s_mobile_2[(s_mobile_2.ovs >0) & (s_mobile_2.ovs <0.10)].og_rev.mean()
sub_rev_3 = og_sub_rev_3 - cost_3
#cost_perc_3 = cost_3/sub_rev_3

p2_b = (clv(og_sub_rev_3,0,0.1,60,0.0176)[-1:]['Month 60']) #before
p2_a = (clv(sub_rev_3,0,0.1,60,0.0112)[-1:]['Month 60']) #after

In [42]:
p2_a - p2_b

Offer CLV
CLV    115.001
Name: Month 60, dtype: float64

### PLAN 3

In [43]:
s_mobile_eqpd = s_mobile.copy()

cust_list = s_mobile_eqpd.customer[(s_mobile_eqpd.representative == 1)  & (s_mobile_eqpd.eqpdays > 365)]
resp_list = cust_list.sample(frac=0.1, random_state = 1234)
resp_list = resp_list.to_list()

In [44]:
s_mobile_eqpd = s_mobile_eqpd.reset_index(drop = True)

for i in range(s_mobile_eqpd.shape[0]):
    if s_mobile_eqpd['customer'][i] in resp_list:
        s_mobile_eqpd['eqpdays'][i] = 0
        

Look at only among people that we actually mailed, so those in `cust_list`. Only looking at those (10%) that actually responded

In [45]:
s_mobile_eqp = s_mobile_eqpd.query("representative == 1").copy()
s_mobile_eqp["p_eqp"] = lr.predict(s_mobile_eqp)

s_mobile_eqp[s_mobile_eqp['customer'].isin(resp_list)][["churn_yes", "p_eqp"]].agg(
    ["count", "mean"]
).round(4)

,churn_yes,p_eqp
count,2188.0000,2188.0000
mean,0.0192,0.0096


In [46]:
eqp_rev = s_mobile_eqp[s_mobile_eqp['customer'].isin(resp_list)].revenue.mean()
#eqp_rev

In [47]:
# eqp days
p3_b = (clv(eqp_rev,0,0.1,60,0.0192)[-1:]['Month 60']) #before #before
p3_a = (clv(eqp_rev,0,0.1,60,0.0096)[-1:]['Month 60']) #after

In [48]:
p3_a - p3_b

Offer CLV
CLV    352.761
Name: Month 60, dtype: float64